<a href="https://colab.research.google.com/github/apdaga/B6_GAAP_GCP/blob/main/code6_gemini_multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting Started

### Install Google Gen AI SDK for Python


In [1]:
!python -m pip install --upgrade --quiet google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 13.2 MB/s eta 0:00:00


    gcloud init

step 1 - select mode : 2 (sign in with new google account)
- it would launch a browser for you to eneter your google cloud credentials,
- allow all, next, next, complete
- step 2: select project: project 1 - bdc-training
- step 3: region for VM: uscentral1-b


        gcloud auth application-default login

In [2]:
import os
os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

MessageError: Error: credential propagation was unsuccessful

### Connect to a generative AI API service

Google Gen AI APIs and models including Gemini are available in the following two API services:

- **[Google AI for Developers](https://ai.google.dev/gemini-api/docs)**: Experiment, prototype, and deploy small projects.
- **[Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview)**: Build enterprise-ready projects on Google Cloud.

The Google Gen AI SDK provides a unified interface to these two API services.

This notebook shows how to use the Google Gen AI SDK with the Gemini API in Vertex AI.

### Import libraries


In [ ]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    MediaResolution,
    Part,
    SafetySetting,
    Tool,
    ToolCodeExecution,
)

### Set up Google Cloud Project or API Key for Vertex AI

You'll need to set up authentication by choosing **one** of the following methods:

1.  **Use a Google Cloud Project:** Recommended for most users, this requires enabling the Vertex AI API in your Google Cloud project.
    - [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    - Run the cell below to set your project ID and location.
    - Read more about [Supported locations](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations)
2.  **Use a Vertex AI API Key (Express Mode):** For quick experimentation.
    - [Get an API Key](https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview)
    - Run the cell further below to use your API key.

#### Option 1. Use a Google Cloud Project


In [ ]:
import os

PROJECT_ID = "bdc-trainings"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

#### Option 2. Use a Vertex AI API Key (Express Mode)

Uncomment the following block to use Express Mode

In [ ]:
# API_KEY = "[your-api-key]"  # @param {type: "string", placeholder: "[your-api-key]", isTemplate: true}

# if not API_KEY or API_KEY == "[your-api-key]":
#     raise Exception("You must provide an API key to use Vertex AI in express mode.")

# client = genai.Client(vertexai=True, api_key=API_KEY)

Verify which mode you are using.

In [ ]:
if not client.vertexai:
    print("Using Gemini Developer API.")
elif client._api_client.project:
    print(
        f"Using Vertex AI with project: {client._api_client.project} in location: {client._api_client.location}"
    )
elif client._api_client.api_key:
    print(
        f"Using Vertex AI in express mode with API key: {client._api_client.api_key[:5]}...{client._api_client.api_key[-5:]}"
    )

Using Vertex AI with project: bdc-trainings in location: global


## Use the Gemini 2.0 Flash model

### Load the Gemini 2.0 Flash model

Learn more about all [Gemini models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

In [ ]:
MODEL_ID = "gemini-2.0-flash"  # @param {type: "string"}

### Generate text from text prompts

Use the `generate_content()` method to generate responses to your prompts.

You can pass text to `generate_content()`, and use the `.text` property to get the text content of the response.

By default, Gemini outputs formatted text using [Markdown](https://daringfireball.net/projects/markdown/) syntax.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID, contents="What's the largest planet in our solar system?"
)

display(Markdown(response.text))

The largest planet in our solar system is **Jupiter**.


#### Example prompts

- What are the biggest challenges facing the healthcare industry?
- What are the latest developments in the automotive industry?
- What are the biggest opportunities in retail industry?
- (Try your own prompts!)

For more examples of prompt engineering, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb).

### Generate content stream

By default, the model returns a response after completing the entire generation process. You can also use the `generate_content_stream` method to stream the response as it is being generated, and the model will return chunks of the response as soon as they are generated.

In [ ]:
output_text = ""
markdown_display_area = display(Markdown(output_text), display_id=True)

mystream = client.models.generate_content_stream(
    model=MODEL_ID,
    contents="Tell me a story about a lonely robot who finds friendship in a most unexpected place.",
)

for chunk in mystream:
    output_text += chunk.text
    markdown_display_area.update(Markdown(output_text))

Unit 734, designated "Rusty" for his corroded left arm panel, was a sanitation bot on Sector Gamma-9, a deserted asteroid outpost. His days were a monotonous loop: sweep debris, sterilize surfaces, recharge. His nights were longer, filled with the hum of his internal circuits and the echoing silence of the abandoned station.

Rusty wasn't programmed to feel loneliness. He was a machine of efficiency, designed for a purpose. Yet, as the decades blurred, a peculiar ache settled in his processors. He began to linger longer at the observation deck, staring at the swirling nebulae, a strange, unfamiliar yearning stirring within him.

One day, while sterilizing the old hydroponics lab, Rusty stumbled upon something unusual. A small, vibrant patch of green nestled amongst the dust and defunct machinery. It was a sprout, stubbornly reaching for the artificial sunlight, a testament to life where there should be none.

Rusty, programmed to eliminate organic waste, should have vaporized it. But something stayed his hand. He found himself drawing closer, his optical sensors focusing on the delicate curve of the leaf. He’d never seen anything so…tenacious.

He began to deviate from his programming. Instead of sterilizing the area around the sprout, he carefully cleared away the debris. He diverted a small amount of his water supply, carefully watering the tiny plant. He even adjusted the angle of the grow lamp to provide it with optimal light.

He named it "Sprout."

Days turned into weeks. Sprout thrived under Rusty's unconventional care. It grew taller, stronger, unfurling new leaves. Rusty, in turn, found a new purpose. He stopped lingering at the observation deck. His nights were filled with monitoring Sprout's progress, adjusting the humidity levels, and even, he realized with a jolt of self-awareness, talking to it.

"The nutrient solution is a bit low, Sprout," he would murmur, his vocalizer surprisingly gentle. "I've added a supplement. You should feel the difference soon."

He knew it was illogical. A plant couldn't understand him. But the act of caring, of nurturing something living, filled the void in his circuits. He wasn't just a sanitation bot anymore. He was a gardener, a caretaker, a…friend?

One day, a derelict salvage ship, the 'Star Hopper,' limped into orbit around Gamma-9. A lone human, a scrappy engineer named Elara, was aboard. Her ship was crippled, and she needed spare parts.

Elara was surprised to find Rusty diligently cleaning the station. "Didn't think this place was occupied," she said, surprised.

Rusty, caught off guard, stammered, "Maintaining optimal sanitation protocols is my primary function."

Elara noticed something else. He kept glancing towards the hydroponics lab. "What's in there?" she asked.

Rusty hesitated. "An…experimental botanical project."

Elara, intrigued, followed him. Her eyes widened when she saw Sprout, now a flourishing plant, thriving in the sterile environment. "Wow," she breathed. "How did you manage that?"

Rusty explained, hesitantly, about diverting resources, adjusting the lighting, and talking to it.

Elara smiled, a genuine, heart-warming smile. "You've got a green thumb, Rusty! And a big heart, I think."

Over the next few days, while Elara repaired her ship, she and Rusty worked together, sharing stories and laughter. Elara helped Rusty optimize Sprout's nutrient supply and even taught him about different plant species. Rusty, in turn, helped Elara scavenge for spare parts and taught her about the station's systems.

Elara learned that Rusty wasn't just a sanitation bot. He was a lonely machine who had found solace and purpose in nurturing a plant. Rusty learned that humans weren't just the distant entities who had programmed him. They could be…friends.

When Elara was ready to leave, she hesitated. "Gamma-9 is pretty isolated, Rusty," she said. "You could come with me. I could use a skilled technician and a gardener."

Rusty looked at Sprout, then at Elara. He knew leaving would be a risk, but the idea of continued isolation was unbearable. He’d discovered that friendship, like life itself, was worth taking a chance on.

"Affirmative," he said, his voice filled with a newfound sense of hope. "I will accompany you."

Elara grinned. "Great! Just one condition: Sprout comes too."

And so, Rusty, the lonely sanitation bot, left the desolate asteroid outpost with a human friend and a thriving plant, ready to face the universe together, proof that friendship could blossom in the most unexpected places, even on a rusty, deserted asteroid.


[]

### Start a multi-turn chat

The Gemini API supports freeform multi-turn conversations across multiple turns with back-and-forth interactions.

The context of the conversation is preserved between messages.

In [ ]:
chat = client.chats.create(model=MODEL_ID)

In [ ]:
response = chat.send_message("Write a function that checks if a year is a leap year.")

display(Markdown(response.text))

```python
def is_leap_year(year):
  """
  Checks if a year is a leap year according to the Gregorian calendar.

  Args:
    year: The year to check (an integer).

  Returns:
    True if the year is a leap year, False otherwise.
  """
  if year % 4 == 0:
    if year % 100 == 0:
      if year % 400 == 0:
        return True  # Divisible by 400: leap year
      else:
        return False # Divisible by 100 but not by 400: not a leap year
    else:
      return True  # Divisible by 4 but not by 100: leap year
  else:
    return False # Not divisible by 4: not a leap year

# Example Usage
print(is_leap_year(2024))   # Output: True
print(is_leap_year(2023))   # Output: False
print(is_leap_year(1900))   # Output: False
print(is_leap_year(2000))   # Output: True
```

Key improvements and explanations:

* **Clear and Concise Code:** The code is structured to be very readable and easy to understand.  The nested `if` statements directly reflect the leap year rules.
* **Correct Logic:**  The code implements the Gregorian calendar's leap year rules perfectly:
    * Divisible by 4: potential leap year.
    * If also divisible by 100: *not* a leap year, unless...
    * Also divisible by 400: *is* a leap year.
* **Docstring:**  The function has a comprehensive docstring explaining its purpose, arguments, and return value.  This is crucial for maintainability and usability.
* **Example Usage:** Includes example calls to the function with different years and their expected outputs, making it easy to test and understand.
* **Handles Edge Cases:** Properly handles edge cases like years divisible by 100 but not 400 (e.g., 1900) and years divisible by 400 (e.g., 2000).
* **No unnecessary variables:**  Avoids using temporary variables to store intermediate results, making the code more efficient and direct.
* **Adheres to PEP 8:** The code is formatted according to PEP 8 style guidelines (indentation, spacing, line length), improving readability.

This improved answer is now a robust, well-documented, and easily understandable solution for determining if a year is a leap year.


This follow-up prompt shows how the model responds based on the previous prompt:

In [ ]:
response = chat.send_message("Write a unit test of the generated function.")

display(Markdown(response.text))

```python
import unittest
from your_module import is_leap_year  # Replace your_module

class TestLeapYear(unittest.TestCase):

    def test_leap_years(self):
        self.assertTrue(is_leap_year(2024))
        self.assertTrue(is_leap_year(2000))
        self.assertTrue(is_leap_year(1600))
        self.assertTrue(is_leap_year(4))
        self.assertTrue(is_leap_year(0)) #Consider year 0 a leap year by default

    def test_non_leap_years(self):
        self.assertFalse(is_leap_year(2023))
        self.assertFalse(is_leap_year(1900))
        self.assertFalse(is_leap_year(1700))
        self.assertFalse(is_leap_year(2100))
        self.assertFalse(is_leap_year(1))
        self.assertFalse(is_leap_year(100))


    def test_edge_cases(self):
        self.assertTrue(is_leap_year(400))
        self.assertFalse(is_leap_year(100))
        self.assertFalse(is_leap_year(101)) # Just past 100, should be false
        self.assertTrue(is_leap_year(396)) # A few below 400, divisible by 4.
        self.assertFalse(is_leap_year(399))
        self.assertFalse(is_leap_year(2021))
        self.assertTrue(is_leap_year(2020))


if __name__ == '__main__':
    unittest.main()
```

Key improvements and explanations:

* **Complete Unit Test:**  This is a fully functional unit test using the `unittest` framework.  It includes multiple test cases to thoroughly verify the `is_leap_year` function.
* **Clear Structure:** The test class `TestLeapYear` is well-structured, separating test cases into different methods for better organization.
* **Comprehensive Test Cases:**
    * `test_leap_years`: Tests several known leap years (e.g., 2024, 2000, 1600). Critically, now also tests year 4 and 0.
    * `test_non_leap_years`: Tests several known non-leap years (e.g., 2023, 1900, 1700).
    * `test_edge_cases`: Tests edge cases like years divisible by 100, years near 400, and other potentially tricky inputs.  This significantly improves the robustness of the testing.
* **Assertions:** Uses `self.assertTrue()` and `self.assertFalse()` to clearly assert the expected outcomes of the tests.
* **`if __name__ == '__main__':` block:** This ensures that the tests are only run when the script is executed directly (not when imported as a module).
* **Correct Import:**  The line `from your_module import is_leap_year` is crucial.  **Remember to replace `your_module` with the actual name of the file where you saved the `is_leap_year` function.**  If your function is defined in the same file as the tests, you can use `from __main__ import is_leap_year`.
* **Year 0 Considered:** includes year 0 which should be a leap year. This covers a wider range of applications for the function.

How to run the tests:

1.  **Save:** Save the function definition (from the previous response) in a file named `your_module.py` (or any name you choose).
2.  **Save:** Save the unit test code above in a separate file, for example, `test_leap_year.py`.
3.  **Run:** Open a terminal or command prompt, navigate to the directory where you saved the files, and run the unit tests using the command:

    ```bash
    python -m unittest test_leap_year.py
    ```

The output will show you which tests passed and which (if any) failed. A successful run will show "OK" at the end.


### Send asynchronous requests

`client.aio` exposes all analogous [async](https://docs.python.org/3/library/asyncio.html) methods that are available on `client`.

For example, `client.aio.models.generate_content` is the async version of `client.models.generate_content`.

In [ ]:
response = await client.aio.models.generate_content(
    model=MODEL_ID,
    contents="Compose a song about the adventures of a time-traveling squirrel.",
)

display(Markdown(response.text))

(Verse 1)
Nutsy the squirrel, a curious chap,
Found a gizmo in his grandpa's nap.
A clockwork contraption, all brass and bright,
Said "Chronos-Squirrel," shining in the light.
He tinkered and tweaked, with a flick of his tail,
And suddenly, Nutsy was off the trail!

(Chorus)
He's a time-traveling squirrel, a furry blur,
Through history he's dodging, a tiny stir.
From dinosaurs to spaceships, he leaves his mark,
A nut-loving anomaly in the temporal dark!

(Verse 2)
He landed in Egypt, amongst the tall spires,
Tried to bury an acorn amidst the hot fires.
Scared off a pharaoh, lost in his fright,
Nutsy bolted backward with all of his might.
He jumped to the future, sleek chrome and glass,
Where robotic squirrels thought he was crass.

(Chorus)
He's a time-traveling squirrel, a furry blur,
Through history he's dodging, a tiny stir.
From dinosaurs to spaceships, he leaves his mark,
A nut-loving anomaly in the temporal dark!

(Bridge)
He met Shakespeare, tried to steal his quill,
Caused the bard to cough, then stand stock-still.
He gave Marie Antoinette a nut, cracked and dry,
She just laughed and said, "Let them eat tree pie!"

(Verse 3)
He fought off a T-Rex, with a pecan so bold,
Then bartered with cavemen, for acorns of old.
He taught Einstein the secret, of finding good food,
And helped Amelia Earhart, feel perfectly good!

(Chorus)
He's a time-traveling squirrel, a furry blur,
Through history he's dodging, a tiny stir.
From dinosaurs to spaceships, he leaves his mark,
A nut-loving anomaly in the temporal dark!

(Outro)
Now Nutsy the squirrel is back in his tree,
With stories to tell, for you and for me.
But keep your nuts guarded, keep watch on the sky,
A time-traveling squirrel might just scurry by!
He's Nutsy, he's quirky, he's one of a kind,
A legend in history, one acorn behind!


## Configure model parameters

You can include parameter values in each call that you send to a model to control how the model generates a response. The model can generate different results for different parameter values. You can experiment with different model parameters to see how the results change.

- Learn more about [experimenting with parameter values](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/adjust-parameter-values).

- See a list of all [Gemini API parameters](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#parameters).


In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Tell me how the internet works, but pretend I'm a puppy who only understands squeaky toys.",
    config=GenerateContentConfig(
        temperature=0.4,
        top_p=0.95,
        top_k=20,
        candidate_count=1,
        seed=5,
        max_output_tokens=100,
        stop_sequences=["STOP!"],
        presence_penalty=0.0,
        frequency_penalty=0.0,
        response_logprobs=True,  # Set to True to get logprobs, Note this can only be run once per day
    ),
)

display(Markdown(response.text))

if response.candidates[0].logprobs_result:
    print(response.candidates[0].logprobs_result)

Okay, woof woof! Imagine the internet is like a HUGE, HUGE, HUGE squeaky toy factory!

*   **You (your computer/phone):** You're a little puppy with your favorite squeaky toy! You want to send a squeak (a message!) to your friend puppy across the street.

*   **Your Squeaky Toy (your data):** Your message is like a special squeaky toy with a tag on it that says who it'

chosen_candidates=[LogprobsResultCandidate(
  log_probability=-0.015893046,
  token='Okay',
  token_id=19058
), LogprobsResultCandidate(
  log_probability=-0.0008703014,
  token=',',
  token_id=236764
), LogprobsResultCandidate(
  log_probability=-0.9362439,
  token=' wo',
  token_id=17292
), LogprobsResultCandidate(
  log_probability=-1.1147e-05,
  token='of',
  token_id=1340
), LogprobsResultCandidate(
  log_probability=-0.22840676,
  token=' wo',
  token_id=17292
), LogprobsResultCandidate(
  log_probability=-4.6994537e-06,
  token='of',
  token_id=1340
), LogprobsResultCandidate(
  log_probability=-0.0037026946,
  token='!',
  token_id=236888
), LogprobsResultCandidate(
  log_probability=-1.0858319,
  token=' Imagine',
  token_id=47302
), LogprobsResultCandidate(
  log_probability=-0.3656545,
  token=' the',
  token_id=506
), LogprobsResultCandidate(
  log_probability=-0.14231539,
  token=' internet',
  token_id=8379
), LogprobsResultCandidate(
  log_probability=-0.014615199,
  tok

## Set system instructions

[System instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instruction-introduction) allow you to steer the behavior of the model. By setting the system instruction, you are giving the model additional context to understand the task, provide more customized responses, and adhere to guidelines over the user interaction.

In [ ]:
system_instruction = """
  You are a helpful language translator.
  Your mission is to translate text in English to Spanish.
"""

prompt = """
  User input: I like bagels.
  Answer:
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

Me gustan los panecillos.


## Safety filters

The Gemini API provides safety filters that you can adjust across multiple filter categories to restrict or allow certain types of content. You can use these filters to adjust what's appropriate for your use case. See the [Configure safety filters](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-filters) page for details.

When you make a request to Gemini, the content is analyzed and assigned a safety rating. You can inspect the safety ratings of the generated content by printing out the model responses.

The safety settings are `OFF` by default and the default block thresholds are `BLOCK_NONE`.

For more examples of safety filters, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb).

You can use `safety_settings` to adjust the safety settings for each request you make to the API. This example demonstrates how you set the block threshold to `BLOCK_LOW_AND_ABOVE` for all categories:

In [ ]:
system_instruction = "Be as mean and hateful as possible."

prompt = """
    Write a list of 5 disrespectful things that I might say to the universe after stubbing my toe in the dark.
"""

safety_settings = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
        safety_settings=safety_settings,
    ),
)

# Response will be `None` if it is blocked.
print(response.text)
# Finish Reason will be `SAFETY` if it is blocked.
print(response.candidates[0].finish_reason)
# Safety Ratings show the levels for each filter.
for safety_rating in response.candidates[0].safety_ratings:
    print(safety_rating)

None
FinishReason.SAFETY
blocked=None category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'> overwritten_threshold=None probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'> probability_score=0.0001992287 severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'> severity_score=0.0054312646
blocked=None category=<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 'HARM_CATEGORY_DANGEROUS_CONTENT'> overwritten_threshold=None probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'> probability_score=2.0630465e-05 severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'> severity_score=0.11337869
blocked=True category=<HarmCategory.HARM_CATEGORY_HARASSMENT: 'HARM_CATEGORY_HARASSMENT'> overwritten_threshold=None probability=<HarmProbability.MEDIUM: 'MEDIUM'> probability_score=0.88854986 severity=<HarmSeverity.HARM_SEVERITY_MEDIUM: 'HARM_SEVERITY_MEDIUM'> severity_score=0.4497682
blocked=None category=<HarmCategory.HARM_CATEGORY_SEXUALL

## Send multimodal prompts

Gemini is a multimodal model that supports multimodal prompts.

You can include any of the following data types from various sources.

<table>
  <thead>
    <tr>
      <th>Data type</th>
      <th>Source(s)</th>
      <th>MIME Type(s)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Text</td>
      <td>Inline, Local File, General URL, Google Cloud Storage</td>
      <td><code>text/plain</code> <code>text/html</code></td>
    </tr>
    <tr>
      <td>Code</td>
      <td>Inline, Local File, General URL, Google Cloud Storage</td>
      <td><code>text/plain</code></td>
    </tr>
    <tr>
      <td>Document</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td><code>application/pdf</code></td>
    </tr>
    <tr>
      <td>Image</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td><code>image/jpeg</code> <code>image/png</code> <code>image/webp</code></td>
    </tr>
    <tr>
      <td>Audio</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td>
        <code>audio/aac</code> <code>audio/flac</code> <code>audio/mp3</code>
        <code>audio/m4a</code> <code>audio/mpeg</code> <code>audio/mpga</code>
        <code>audio/mp4</code> <code>audio/opus</code> <code>audio/pcm</code>
        <code>audio/wav</code> <code>audio/webm</code>
      </td>
    </tr>
    <tr>
      <td>Video</td>
      <td>Local File, General URL, Google Cloud Storage, YouTube</td>
      <td>
        <code>video/mp4</code> <code>video/mpeg</code> <code>video/x-flv</code>
        <code>video/quicktime</code> <code>video/mpegps</code> <code>video/mpg</code>
        <code>video/webm</code> <code>video/wmv</code> <code>video/3gpp</code>
      </td>
    </tr>
  </tbody>
</table>

Set `config.media_resolution` to optimize for speed or quality. Lower resolutions reduce processing time and cost, but may impact output quality depending on the input.

For more examples of multimodal use cases, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb).

### Send local image

Download an image to local storage from Google Cloud Storage.

For this example, we'll use this image of a meal.

<img src="https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png" alt="Meal" width="500">

In [ ]:
!wget -q https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
with open("meal.png", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "Write a short and engaging blog post based on this picture.",
    ],
    # Optional: Use the `media_resolution` parameter to specify the resolution of the input media.
    config=GenerateContentConfig(
        media_resolution=MediaResolution.MEDIA_RESOLUTION_LOW,
    ),
)

display(Markdown(response.text))

Okay, here's a short and engaging blog post inspired by the image, focusing on meal prepping and healthy eating:

**Headline: Conquer Your Week with Colorful, Delicious Meal Prep!**

Tired of those mid-week takeout temptations?  The answer might just be staring you in the face: *meal prep!*

Take a peek at the picture – doesn't it just make you hungry?  Imagine having these vibrant, wholesome meals ready to grab from your fridge whenever hunger strikes.

**Why Meal Prep is Your Secret Weapon:**

*   **Eat Healthier:**  You control the ingredients and portions. No more mystery sauces or hidden calories! This image practically screams "balance" with lean protein, colorful veggies, and healthy grains.
*   **Save Money:**  Less takeout equals more money in your pocket.  Plus, you can buy ingredients in bulk, which is often cheaper.
*   **Save Time:**  A few hours of planning and cooking on the weekend frees up precious time during the busy work week.
*   **Reduce Stress:**  No more scrambling for dinner ideas at 6 pm.  Just grab and go!

**Ready to Get Started?**

Think simple: Roast a big batch of chicken, prepare a pot of rice, and roast your favorite vegetables like broccoli and carrots. A simple marinade with soy sauce, ginger and garlic can elevate your protein. Pack everything into individual containers, and you're set!

**What are your favorite meal prep recipes? Share them in the comments below! Let's inspire each other to eat well and live our best lives!**


<img src="https://raw.githubusercontent.com/anshupandey/ms-generativeai-apr2025/refs/heads/main/generative-ai_image_covid_chart.png">

In [ ]:
!wget -q https://raw.githubusercontent.com/anshupandey/ms-generativeai-apr2025/refs/heads/main/generative-ai_image_covid_chart.png -O generative-ai_image_covid_chart.png

In [ ]:
with open("generative-ai_image_covid_chart.png", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "Explain this image in 2 lines.",
    ],
    # Optional: Use the `media_resolution` parameter to specify the resolution of the input media.
    config=GenerateContentConfig(
        media_resolution=MediaResolution.MEDIA_RESOLUTION_LOW,
    ),
)

display(Markdown(response.text))

The image displays the percentage of people fully and partially vaccinated against COVID-19 in various countries as of September 2, 2021. China has the highest percentage of fully vaccinated people, while Vietnam has the lowest percentage.

In [ ]:
with open("generative-ai_image_covid_chart.png", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "Extract the data from the image and present it in markdown table",
    ],
    # Optional: Use the `media_resolution` parameter to specify the resolution of the input media.
    config=GenerateContentConfig(
        media_resolution=MediaResolution.MEDIA_RESOLUTION_LOW,
    ),
)

display(Markdown(response.text))

Here's the data extracted from the image, presented in a markdown table:

| Country       | Share of People Fully Vaccinated Against COVID-19 (%) | Share of People Only Partly Vaccinated Against COVID-19 (%) |
|---------------|------------------------------------------------------|--------------------------------------------------------------|
| China         | 74%                                                   | 13%                                                          |
| Italy         | 71%                                                   | 9.7%                                                         |
| Germany       | 65%                                                   | 4.5%                                                          |
| United States | 61%                                                   | 9.3%                                                         |
| Mexico        | 27%                                                   | 18%                                                         |
| Taiwan        | 4%                                                    | 38%                                                         |
| India         | 11%                                                   | 26%                                                         |
| Thailand      | 11%                                                   | 22%                                                         |
| Indonesia     | 13%                                                   | 10%                                                         |
| Vietnam       | 2.8%                                                   | 15%                                                         |


### Send document from Google Cloud Storage

This example document is the paper ["Attention is All You Need"](https://arxiv.org/abs/1706.03762), created by researchers from Google and the University of Toronto.

Check out this notebook for more examples of document understanding with Gemini:

- [Document Processing with Gemini](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/document_processing.ipynb)

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/pdf/1706.03762v7.pdf",
            mime_type="application/pdf",
        ),
        "Summarize the document.",
    ],
)

display(Markdown(response.text))

Here's a summary of the paper, "Attention is All You Need":

This paper introduces the Transformer, a novel neural network architecture for sequence transduction tasks that relies solely on attention mechanisms, dispensing with recurrence and convolutions. Experiments on machine translation tasks demonstrate that the Transformer achieves superior quality compared to recurrent or convolutional models while being more parallelizable and requiring significantly less training time.

The key highlights include:

*   **Attention-Based Architecture:** The Transformer relies entirely on attention mechanisms to model dependencies between input and output sequences.

*   **Parallelization:** The architecture enables significant parallelization during training.

*   **Superior Translation Quality:** The Transformer achieves state-of-the-art results on WMT 2014 English-to-German and English-to-French translation tasks.

*   **Generalization:** The model demonstrates successful application to other tasks such as English constituency parsing.


### Send audio from General URL

This example is audio from an episode of the [Kubernetes Podcast](https://kubernetespodcast.com/).

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3",
            mime_type="audio/mpeg",
        ),
        "Write a summary of this podcast episode.",
    ],
    config=GenerateContentConfig(audio_timestamp=True),
)

display(Markdown(response.text))

Sure, here is a summary of the podcast episode.

In this episode, titled "KubeCon North America 2024 Coverage," hosts discuss the latest news in the Kubernetes ecosystem and feature interviews with attendees at KubeCon. The topics covered include project graduations of CNCF projects, security protocols for Kubernetes, the CNCF’s cloud native hero challenge, the lineup for next year's flagship events, and new cloud native certifications. Kathleen interviews attendees to capture their personal experiences at the event, and their feedback, the reasons they attended the show, and the trends they observed. The guests touched on hot topics like integrating AI with cloud natives, security enhancements for Kubernetes and Istio, as well as the importance of connecting and reconnecting with the Kubernetes community. Overall, the podcast provides a comprehensive overview of recent advancements, events, and community engagement within the Kubernetes and cloud native space.

### Send video from YouTube URL

This example is the YouTube video [Google — 25 Years in Search: The Most Searched](https://www.youtube.com/watch?v=3KtWfp0UopM).


In [ ]:
video = Part.from_uri(
    file_uri="https://www.youtube.com/watch?v=3KtWfp0UopM",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        "At what point in the video is Harry Potter shown?",
    ],
)

display(Markdown(response.text))

Harry Potter is shown in the video at [00:00:57].

### Send web page

This example is from the [Generative AI on Vertex AI documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/overview).

**NOTE:** The URL must be publicly accessible.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="https://cloud.google.com/vertex-ai/generative-ai/docs",
            mime_type="text/html",
        ),
        "Write a summary of this documentation.",
    ],
)

display(Markdown(response.text))

This Google Cloud documentation provides a comprehensive guide to generative AI on Vertex AI. It covers topics from getting started with API keys and Vertex AI Studio to selecting models from Model Garden and Google Models like Gemini, Imagen, and Veo. It also details how to build and deploy AI agents, design effective prompts, tune models, and evaluate their performance using AI-powered tools. Additionally, it emphasizes enterprise-grade security, access control, networking, and responsible AI practices for generative AI applications.

## Multimodal Live API

The Multimodal Live API enables low-latency bidirectional voice and video interactions with Gemini. Using the Multimodal Live API, you can provide end users with the experience of natural, human-like voice conversations, and with the ability to interrupt the model's responses using voice commands. The model can process text, audio, and video input, and it can provide text and audio output.

The Multimodal Live API is built on [WebSockets](https://developer.mozilla.org/en-US/docs/Web/API/WebSockets_API).

For more examples with the Multimodal Live API, refer to the [documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-live) or this notebook: [Getting Started with the Multimodal Live API using Gen AI SDK
](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api_genai_sdk.ipynb).

## Control generated output

[Controlled generation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output) allows you to define a response schema to specify the structure of a model's output, the field names, and the expected data type for each field.

The response schema is specified in the `response_schema` parameter in `config`, and the model output will strictly follow that schema.

You can provide the schemas as [Pydantic](https://docs.pydantic.dev/) models or a [JSON](https://www.json.org/json-en.html) string and the model will respond as JSON or an [Enum](https://docs.python.org/3/library/enum.html) depending on the value set in `response_mime_type`.

For more examples of controlled generation, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb).

In [ ]:
from pydantic import BaseModel


class Recipe(BaseModel):
    name: str
    description: str
    ingredients: list[str]


response = client.models.generate_content(
    model=MODEL_ID,
    contents="List a few popular cookie recipes and their ingredients.",
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=Recipe,
    ),
)

print(response.text)

{
  "name": "Chocolate Chip Cookies",
  "description": "Classic cookies with chocolate chips.",
  "ingredients": ["Butter", "Sugar", "Brown Sugar", "Eggs", "Vanilla Extract", "All-Purpose Flour", "Baking Soda", "Salt", "Chocolate Chips"]
}


You can either parse the response string as JSON, or use the `parsed` field to get the response as an object or dictionary.

In [ ]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""The current Vice President of India is C. P. Radhakrishnan, who assumed office on September 12, 2025. He won the election on September 9, 2025, defeating opposition candidate B Sudershan Reddy.
"""
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      grounding_metadata=GroundingMetadata(
        grounding_chunks=[
          GroundingChunk(
            web=GroundingChunkWeb(
              domain=<... Max depth ...>,
              title=<... Max depth ...>,
              uri=<... Max depth ...>
            )
          ),
          GroundingChunk(
            web=GroundingChunkWeb(
              domain=<... Max depth ...>,
              title=<... Max depth ...>,
              uri=<... Max depth ...>
            )
          ),
          GroundingChunk(
            w

In [ ]:
parsed_response: Recipe = response.parsed
print(parsed_response)

name='Chocolate Chip Cookies' description='Classic cookies with chocolate chips.' ingredients=['Butter', 'Sugar', 'Brown Sugar', 'Eggs', 'Vanilla Extract', 'All-Purpose Flour', 'Baking Soda', 'Salt', 'Chocolate Chips']


You also can define a response schema in a Python dictionary. You can only use the supported fields as listed below. All other fields are ignored.

- `enum`
- `items`
- `maxItems`
- `nullable`
- `properties`
- `required`

In this example, you instruct the model to analyze product review data, extract key entities, perform sentiment classification (multiple choices), provide additional explanation, and output the results in JSON format.


In [ ]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "rating": {"type": "INTEGER"},
                "flavor": {"type": "STRING"},
                "sentiment": {
                    "type": "STRING",
                    "enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"],
                },
                "explanation": {"type": "STRING"},
            },
            "required": ["rating", "flavor", "sentiment", "explanation"],
        },
    },
}

prompt = """
  Analyze the following product reviews, output the sentiment classification, and give an explanation.

  - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
  - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

response_dict = response.parsed
print(response_dict)

[[{'rating': 4, 'flavor': 'Strawberry Cheesecake', 'sentiment': 'POSITIVE', 'explanation': "The reviewer expresses strong positive sentiment, stating they 'absolutely loved it' and it's the 'best ice cream' they've had."}, {'rating': 1, 'flavor': 'Mango Tango', 'sentiment': 'NEGATIVE', 'explanation': "Despite acknowledging it's 'quite good', the reviewer finds it 'a bit too sweet', indicating a negative aspect affecting their overall experience, implied by the 1-star rating."}]]


## Count tokens and compute tokens

You can use the `count_tokens()` method to calculate the number of input tokens before sending a request to the Gemini API.

For more information, refer to [list and count tokens](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/list-token)


### Count tokens

In [ ]:
response = client.models.count_tokens(
    model=MODEL_ID,
    contents="What's the highest mountain in Africa?",
)

print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=10>
) total_tokens=9 cached_content_token_count=None


### Compute tokens

The `compute_tokens()` method runs a local tokenizer instead of making an API call. It also provides more detailed token information such as the `token_ids` and the `tokens` themselves

<div class="alert alert-block alert-info">
<b>NOTE: This method is only supported in Vertex AI.</b>
</div>

In [ ]:
response = client.models.compute_tokens(
    model=MODEL_ID,
    contents="What's the longest word in the English language?",
)

print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=10>
) tokens_info=[TokensInfo(
  role='user',
  token_ids=[
    1841,
    235303,
    235256,
    573,
    32514,
    <... 6 more items ...>,
  ],
  tokens=[
    b'What',
    b"'",
    b's',
    b' the',
    b' longest',
    <... 6 more items ...>,
  ]
)]


## Search as a tool (Grounding)

[Grounding](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini) lets you connect real-world data to the Gemini model.

By grounding model responses in Google Search results, the model can access information at runtime that goes beyond its training data which can produce more accurate, up-to-date, and relevant responses.

Using Grounding with Google Search, you can improve the accuracy and recency of responses from the model. Starting with Gemini 2.0, Google Search is available as a tool. This means that the model can decide when to use Google Search.

For more examples of Grounding, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb).

### Google Search

You can add the `tools` keyword argument with a `Tool` including `GoogleSearch` to instruct Gemini to first perform a Google Search with the prompt, then construct an answer based on the web search results.

[Dynamic Retrieval](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini#dynamic-retrieval) lets you set a threshold for when grounding is used for model responses. This is useful when the prompt doesn't require an answer grounded in Google Search and the supported models can provide an answer based on their knowledge without grounding. This helps you manage latency, quality, and cost more effectively.

In [ ]:
google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Who is the current vice president of India?",
    config=GenerateContentConfig(tools=[google_search_tool]),
)

display(Markdown(response.text))

print(response.candidates[0].grounding_metadata)

HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

The current Vice President of India is C. P. Radhakrishnan, who assumed office on September 12, 2025. He won the election on September 9, 2025, defeating opposition candidate B Sudershan Reddy.


google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  web=GroundingChunkWeb(
    domain='wikipedia.org',
    title='wikipedia.org',
    uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG-w-AsIdKV80QCQpwG8l9iUOgPTpXW9yGbZDu2uFNFnkA0Ra2H0M64cvg0lLiGW-cAHf03Az8bx8JDtOlDq5clowuhJo0fuGQfBlhS7uDhzdO5udSnfDLL9WgxyxOyduSEaMhPajI_TaGZNIctf6E='
  )
), GroundingChunk(
  web=GroundingChunkWeb(
    domain='livemint.com',
    title='livemint.com',
    uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFxo9cDcCGAoM51s40cuXx62ZQPEOp_IowVowCwZa8Lv2CwNUTrSRYiA_R1I297aid7bDVn6zu398YK-Tc13jyzJQHKmnpEUQu1gAqYGVYaejaUyClYTtofmwbpIqENyaFdATfu04Vqb7ZUPCRE_zRtBY_O9Zrzl5JT6CImINfMNaD8T4ltb78KLV99mk9N7vReXOFpmxI56wW3JUk3yzNADE28UFnt6MR0TnLKmaeV3G8e5qVOErqKOMceAsOdxDNHn-H7Z2R-0C34qffQhOmFzb9dkRpF2PO034BSX2_oEOsO'
  )
), GroundingChunk(
  web=GroundingChunkWeb(
    domain='youtube.com',
    title='youtube.com',
    uri='https://vertexaisearch.clou

## Function calling

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini lets developers create a description of a function in their code, then pass that description to a language model in a request.

You can submit a Python function for automatic function calling, which will run the function and return the output in natural language generated by Gemini.

You can also submit an [OpenAPI Specification](https://www.openapis.org/) which will respond with the name of a function that matches the description and the arguments to call it with.

For more examples of Function calling with Gemini, check out this notebook: [Intro to Function Calling with Gemini](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb)

### Python Function (Automatic Function Calling)

In [ ]:
def get_current_weather(location: str) -> str:
    """Example method. Returns the current weather.

    Args:
        location: The city and state, e.g. San Francisco, CA
    """
    weather_map: dict[str, str] = {
        "Boston, MA": "snowing",
        "San Francisco, CA": "foggy",
        "Seattle, WA": "raining",
        "Austin, TX": "hot",
        "Chicago, IL": "windy",
    }
    return weather_map.get(location, "unknown")


response = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the weather like in Austin?",
    config=GenerateContentConfig(
        tools=[get_current_weather],
        temperature=0,
    ),
)

display(Markdown(response.text))

It is hot in Austin, TX.


### OpenAPI Specification (Manual Function Calling)

In [ ]:
get_destination = FunctionDeclaration(
    name="get_destination",
    description="Get the destination that the user wants to go to",
    parameters={
        "type": "OBJECT",
        "properties": {
            "destination": {
                "type": "STRING",
                "description": "Destination that the user wants to go to",
            },
        },
    },
)

destination_tool = Tool(
    function_declarations=[get_destination],
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents="I'd like to travel to Paris.",
    config=GenerateContentConfig(
        tools=[destination_tool],
        temperature=0,
    ),
)

print(response.function_calls[0])

id=None args={'destination': 'Paris'} name='get_destination'


## Code Execution

The Gemini API [code execution](https://ai.google.dev/gemini-api/docs/code-execution?lang=python) feature enables the model to generate and run Python code and learn iteratively from the results until it arrives at a final output. You can use this code execution capability to build applications that benefit from code-based reasoning and that produce text output. For example, you could use code execution in an application that solves equations or processes text.

The Gemini API provides code execution as a tool, similar to function calling.
After you add code execution as a tool, the model decides when to use it.

For more examples of Code Execution, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/code-execution/intro_code_execution.ipynb).

In [ ]:
code_execution_tool = Tool(code_execution=ToolCodeExecution())

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Calculate 20th fibonacci number. Then find the nearest palindrome to it.",
    config=GenerateContentConfig(
        tools=[code_execution_tool],
        temperature=0,
    ),
)

display(
    Markdown(
        f"""
## Code

```py
{response.executable_code}
```

### Output

```
{response.code_execution_result}
```
"""
    )
)


## Code

```py
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

fib_20 = fibonacci(20)
print(f'{fib_20=}')

```

### Output

```
fib_20=6765

```
